In [1]:
import pandas as pd

In [6]:
#make a dataframe ignoring the last one from train.txt

In [9]:
#show the head()

#train['ppos'] = train.pos.shift(1)
#train['ppos'] = train['ppos'].fillna('O')

### Write the functions
$P(T{i}|T{i-1})$
$P(W{i}|T{i})$

pos_ppos = {}

for col_name in set(test.ppos):
    sub_frame = train
def Ptt(Ti,prev)

if Ti not in pos_ppos[Tprev] return smooth (.000000001)

In [7]:
#build the trellis

sequence = ['the', 'dog', 'ran','far']

## Evaluate: decode